In [1]:
!pip install youtube-transcript-api
!pip install scrapetube
!pip install pytube
!pip install unidecode
!python -m spacy download pt_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 7.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 8.4 MB/s eta 0:00:00
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-10 00:44:20.106687: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild Tensor

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
import scrapetube
from pytube import YouTube
import pandas as pd

from unidecode import unidecode
import string
import spacy
import nltk
import re

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
nlp = spacy.load('pt_core_news_lg')

In [5]:
videos = list(scrapetube.get_playlist('PLWieWKZeFoVRmUNn9KA2dVpQswo19QkwQ'))

In [6]:
len(videos)

163

In [7]:
videos[0].keys()

dict_keys(['videoId', 'thumbnail', 'title', 'index', 'shortBylineText', 'lengthText', 'navigationEndpoint', 'lengthSeconds', 'trackingParams', 'isPlayable', 'menu', 'thumbnailOverlays', 'videoInfo'])

In [ ]:
transcriptions = list()

for v in videos:

  try:
    video_id = v['videoId']
    video_metadata = YouTube(f'https://www.youtube.com/watch?v={video_id}')
    video_title = video_metadata.title
    views_count = video_metadata.views
    video_description = video_metadata.description
    video_publish_date = video_metadata.publish_date
    transcription = [t['text'] for t in YouTubeTranscriptApi.get_transcript(video_id, languages=['pt'])]

    transcriptions.append([video_id, video_title, video_description, video_publish_date, views_count, transcription])

  except Exception as e:
    print(e)
    continue

In [9]:
len(transcriptions)

144

In [10]:
df = pd.DataFrame(transcriptions, columns=['video_id', 'video_title', 'video_description', 'video_publish_date', 'views_count', 'transcription'])
df.head()

,video_id,video_title,video_description,video_publish_date,views_count,transcription
0,O2PehdBokN8,ÁLVARO MACHADO DIAS e MARTHA GABRIEL - Flow #183,Mais um episódio de dar câimbra no cérebro.\n\...,2023-03-07,88904,"[[Música], [Música], salve salve família bem-v..."
1,A6jUhv0T5Hg,"MIGUEL NICOLELIS, PETRIA CHAVES e OSCAR ULISSE...",Hoje vamos todos nos sentir burros.\n\n~~~~~~~...,2023-03-03,225131,"[[Música], [Música], [Música], agora esse Banc..."
2,bDD6Pj74Jc4,"NANDO VIANA, PATRICK MAIA e BRUNO ROMANO - Flo...",Esses 3 são deveras engraçados.\n\nEspecial do...,2023-03-02,167535,"[[Música], [Música], muito bem muito bem começ..."
3,JW_Bd4Pq6jM,ROSÂNGELA MONTEIRO e BETO RIBEIRO - Flow #180,Os cara do true crime.\n\n~~~~~~~\n\nHIDROMEL\...,2023-03-01,384545,"[[Música], [Música], [Música], salve salve fam..."
4,SEZ-19y06uU,DIGUINHO CORUJA e PALHAÇO AMENDOIM [+ VINHETEI...,Palhaço Coruja e Diguinho Amendoim.\n\n~~~~~~~...,2023-02-23,306237,"[[Música], [Música], [Música], um jardim, salv..."


In [11]:
df['transcription'].explode().value_counts().head(15)

[Música]     1192
[Risadas]    1052
E aí          335
[ __ ]        268
e             237
né            202
é             180
o             169
cara          162
não           138
assim         128
a             123
que           117
hahaha        110
então         102
Name: transcription, dtype: int64

In [12]:
def remove_itens(txt):
  txt = [x for x in txt if x not in ['[Música]', '[Risadas]', '[ __ ]', '[Aplausos]']]

  return txt

In [13]:
df['transcription'] = df['transcription'].apply(remove_itens)

In [14]:
df.head()

,video_id,video_title,video_description,video_publish_date,views_count,transcription
0,O2PehdBokN8,ÁLVARO MACHADO DIAS e MARTHA GABRIEL - Flow #183,Mais um episódio de dar câimbra no cérebro.\n\...,2023-03-07,88904,"[salve salve família bem-vindos a mais um, Flo..."
1,A6jUhv0T5Hg,"MIGUEL NICOLELIS, PETRIA CHAVES e OSCAR ULISSE...",Hoje vamos todos nos sentir burros.\n\n~~~~~~~...,2023-03-03,225131,"[agora esse Banco esse banco é incrível, nunca..."
2,bDD6Pj74Jc4,"NANDO VIANA, PATRICK MAIA e BRUNO ROMANO - Flo...",Esses 3 são deveras engraçados.\n\nEspecial do...,2023-03-02,167535,"[muito bem muito bem começa então tu vai, e aí..."
3,JW_Bd4Pq6jM,ROSÂNGELA MONTEIRO e BETO RIBEIRO - Flow #180,Os cara do true crime.\n\n~~~~~~~\n\nHIDROMEL\...,2023-03-01,384545,"[salve salve família bem-vindos a mais um, Flo..."
4,SEZ-19y06uU,DIGUINHO CORUJA e PALHAÇO AMENDOIM [+ VINHETEI...,Palhaço Coruja e Diguinho Amendoim.\n\n~~~~~~~...,2023-02-23,306237,"[um jardim, salve salve família bem-vindos a m..."


In [15]:
df.to_excel('extractions.xlsx', index=False)

In [16]:
df['transcription'][0][:5]

['salve salve família bem-vindos a mais um',
 'Flow eu sou o Igor e hoje eu vou',
 'conversar com Marta Gabriel Obrigado por',
 'vir aí trazer gigante e Álvaro Machado',
 'Dias família']

In [17]:
def remove_stopwords(text, stop_words=stopwords):

  filtered_text = [word for word in text if word not in stop_words]

  return filtered_text

def remove_punctuation(text): 

  translator = str.maketrans('', '', string.punctuation)

  return text.translate(translator) 

def remove_accented_chars(text):
  
  text = unidecode(text)
  return text

def remove_numbers(text): 

  result = re.sub(r'\d+', '', text)

  return result

def lemma(txt):
  
  doc = nlp(' '.join(txt))
  doc = ' '.join([token.lemma_ for token in doc])

  return doc

def prepare_text(transcription_list):

  return lemma(remove_stopwords(remove_numbers(remove_punctuation(' '.join(transcription_list).lower()))))

In [ ]:
df['clean_transcription'] = df['transcription'].apply(prepare_text)
df['clean_transcription'][0]

In [ ]:
#from nltk.stem import RSLPStemmer
#nltk.download('rslp')
#stemmer = RSLPStemmer()
#[stemmer.stem(x) for x in df['transcription'][0][10].split()]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vetor = CountVectorizer()
x = vetor.fit_transform(df['clean_transcription']).toarray()
x

In [ ]:
len(x[0])

quantidade de views aumenta com o tempo -> corrigir removendo a média